In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<h1> 1 - IOU(box, centroid)  metric should be used </h1>

In [1]:
# !pip install torchvision/

import os
from IPython.display import display as dis
from IPython.display import Image as im
import cv2
import matplotlib.pyplot as plt
import random
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import torch
import time
import torchvision


from torchvision.ops import box_iou

In [3]:
import shutil

shutil.copy("/content/drive/MyDrive/dataset_udacity/box_priors.json", "./")
shutil.copy("/content/drive/MyDrive/dataset_udacity/file_orderer.json", "./")
shutil.copy("/content/drive/MyDrive/dataset_udacity/data.yaml", "./")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/dataset_udacity/box_priors.json'

In [2]:
import torch
from torch import nn
from collections import OrderedDict

"""
- The custom yolo model is implemented as seen in the architecture diagrams
- the parts , as in the diagram are:
  1. backbone
  2. neck
  3-7. heads 1-5

- head numbers are defined as in the architecture diagram
"""

class normed_Conv2d(nn.Conv2d):
  def __init__(self,
               do_norm = True,
               act = "silu",
               **kwargs):
    super(normed_Conv2d, self).__init__(**kwargs)
    self.normed = do_norm
    self.norm_layer = nn.BatchNorm2d(num_features = self.out_channels)

    if act == 'silu':
      self.activation = nn.SiLU()

    elif act =='relu':
      self.activation = nn.ReLU()

    # elif act == 'softmax':
    #   self.activation = nn.

  def forward(self, x):
    x = super(normed_Conv2d, self).forward(x)
    if self.normed:
      x = self.norm_layer(x)
    x = self.activation(x)
    return x




""" NN.SEQUENTIAL CAN BE USED INSTEAD OF MODULELIST, BUT IT TAKES THE SAME TIME AS LOOPING OVER LAYERS IN MODULELIST
DEFINITION SHOULD BE DONE BY MAKING AN ORDEREDDICT OF THE LAYERS"""
class bottleneck(nn.Module):
  def __init__(self,
               in_channels,
               num_1x1_filters_per_rep,
               num_3x3_filters_per_rep,
               num_reps,
               block_number = None,
               downsample = False
               ):
    super(bottleneck, self).__init__()

    self.num_1x1_filters_per_rep = num_1x1_filters_per_rep
    self.num_3x3_filters_per_rep = num_3x3_filters_per_rep
    self.num_reps = num_reps

    layer_block = nn.ModuleList()

    if block_number == None:
        block_number = "NA"

    if downsample:
      padding_3x3 = 0
    else:
      padding_3x3 = 1

    for rep_no in range(self.num_reps):

      rep = OrderedDict()

      rep[f"block_{block_number}_rep_{rep_no}_ConvBNSiLU_1x1"] = normed_Conv2d(
          in_channels =  in_channels,
          out_channels = self.num_1x1_filters_per_rep,
          kernel_size = 1,
          # padding = 0 #same
          )

      rep[f"block_{block_number}_rep_{rep_no}_ConvBNSiLU_3x3"] = normed_Conv2d(
          in_channels = self.num_1x1_filters_per_rep,
          out_channels = self.num_3x3_filters_per_rep,
          kernel_size = 3,
          padding = padding_3x3 # same
          )

      rep = nn.Sequential(rep)

      layer_block.add_module(name = f"block_{block_number}_rep_{rep_no}",
                             module = rep)

      in_channels = self.num_3x3_filters_per_rep

    self.layer_block = layer_block


  def forward(self, x):
    init_layer = True
    for layer in self.layer_block:
      if init_layer:
        x = layer(x)
      else:
        x = layer(x) + x

    return x



class backbone(nn.Module):
  def __init__(self):
    super(backbone, self).__init__()

    self.super_block = OrderedDict()

    ############### INPUT LAYER ################
    self.super_block["input_layer"] = normed_Conv2d(
        in_channels = 3,
        out_channels = 32,
        kernel_size = 3
    )
    ############################################

    ######## FIRST CONV ONLY BOTTLENECK ########
    self.super_block["ConvBNSiLU_0"] = normed_Conv2d(
        in_channels = 32,
        out_channels = 32,
        kernel_size = 3
    )

    self.super_block["ConvBSSiLU_1"] = normed_Conv2d(
        in_channels = 32,
        out_channels = 64,
        kernel_size = 3,
        stride = 2,
        padding = 1
    )
    ############################################

    #SEQUENTIAL REPETITVE BOTTLENECK SEQUENCE ##

    self.super_block["bottleneck_0"] = bottleneck(
        in_channels = 64,
        num_1x1_filters_per_rep = 32,
        num_3x3_filters_per_rep = 64,
        num_reps = 2,
        block_number = 0,
        downsample = True,
    )

    self.super_block["ConvBSSiLU_2"] = normed_Conv2d(
        in_channels = 64,
        out_channels = 128,
        kernel_size = 3,
        stride = 2,
        padding = 1
    )

    self.super_block["bottleneck_1"] = bottleneck(
        in_channels = 128,
        num_1x1_filters_per_rep = 64,
        num_3x3_filters_per_rep = 128,
        num_reps = 6,
        block_number = 1,
        downsample = True
    )

    self.super_block["ConvBNSiLU_3"] = normed_Conv2d(
        in_channels = 128,
        out_channels = 256,
        kernel_size = 3,
        stride = 2,
        padding = 1
    )

    self.super_block["bottleneck_2"] = bottleneck(
        in_channels = 256,
        num_1x1_filters_per_rep = 128,
        num_3x3_filters_per_rep = 256,
        num_reps = 8,
        block_number = 2,
        downsample = True

    )

    self.super_block["ConvBNSiLU_4"] = normed_Conv2d(
        in_channels = 256,
        out_channels = 512,
        kernel_size = 3,
        stride = 2,
        padding = 1,
    )

    self.super_block["bottleneck_3"] = bottleneck(
        in_channels = 512,
        num_1x1_filters_per_rep = 256,
        num_3x3_filters_per_rep = 512,
        num_reps = 8,
        block_number = 3,
        downsample = True
    )
    ##########################################

    self.super_block = nn.Sequential(self.super_block)

  def forward(self, x):
    x = self.super_block(x)

    return x


class neck(nn.Module):
  def __init__(self):
    super(neck, self).__init__()

    self.layers = nn.ModuleDict()

    self.layers.add_module( name = "ConvBNSiLU_0",
                           module = normed_Conv2d(
                               in_channels = 512,
                               out_channels = 1024,
                               kernel_size = 3,
                               stride = 2,
                               padding = 1
                           ))

    self.layers.add_module( name = "bottleneck_0",
                           module = bottleneck(
                               in_channels = 1024,
                               num_1x1_filters_per_rep = 512,
                               num_3x3_filters_per_rep = 1024,
                               num_reps = 4,
                               block_number = 0
                           ))

    self.layers.add_module( name = "ConvBNSiLU_1",
                           module = normed_Conv2d(
                               in_channels = 1024,
                               out_channels = 512,
                               kernel_size = 3,
                               stride = 2,
                               padding = 1
                           ))

    self.layers.add_module( name = "bottleneck_1",
                           module = bottleneck(
                               in_channels = 512,
                               num_1x1_filters_per_rep = 256,
                               num_3x3_filters_per_rep = 512,
                               num_reps = 4,
                               block_number = 1
                           ))

  def forward(self, x):
    self.input = x
    self.outputs = OrderedDict()
    for name, layer in self.layers.items():
      x = layer(x)
      self.outputs[name] = x

    return self.outputs.copy()

class head_downsample(nn.Module):
  def __init__(self,
               in_channels):
    super(head_downsample, self).__init__()

    self.head = OrderedDict()

    self.in_channels = in_channels

    self.head["bottleneck_0"] = bottleneck(
        in_channels = self.in_channels,
        num_1x1_filters_per_rep = 256,
        num_3x3_filters_per_rep = 512,
        num_reps = 1
    )

    self.head["ConvBNSiLU_0"] = normed_Conv2d(
        do_norm = False,
        in_channels = 512,
        out_channels = 80,
        kernel_size = 3,
        stride = 2,
        padding = 1
    )

    self.head = nn.Sequential(self.head)

  def forward(self, x1, x2):
    x = torch.concat([x1, x2], axis = 1)

    x = self.head(x)

    return x

class head_upsample(nn.Module):
  def __init__(self,
               in_channels):
    super(head_upsample, self).__init__()

    self.upsampler = nn.Upsample(
        scale_factor = 2,
        mode = "nearest",
    )

    self.in_channels = in_channels

    self.head = normed_Conv2d(
        do_norm = False,
        in_channels = self.in_channels,
        out_channels = 80,
        kernel_size = 3,
        stride = 2,
        padding = 1
    )

  def forward(self, x1, x2):
    x2  = self.upsampler(x2)

    x = torch.concat([x1, x2], axis = 1)

    x = self.head(x)

    return x


class head_3(nn.Module):
  def __init__(self):
    super(head_3, self).__init__()

    self.head = normed_Conv2d(
        do_norm = False,
        in_channels = 512,
        out_channels = 80,
        kernel_size = 4,
        # stride = 2,
        # padding = 1
    )

  def forward(self, x):
    x = self.head(x)

    return x


class my_yolo(nn.Module):
  def __init__(self):
    super(my_yolo, self).__init__()

    self.backbone = backbone()

    self.neck = neck()

    self.heads = nn.ModuleDict()

    self.heads.add_module(
        name = "head_1",
        module = head_downsample(
            in_channels = 2048
        )
    )

    self.heads.add_module(
        name = "head_2",
        module = head_downsample(
            in_channels = 1024
        )
    )

    self.heads.add_module(
        name = "head_3",
        module = head_3()
    )

    self.heads.add_module(
        name = "head_4",
        module = head_upsample(
            in_channels = 1536
        )
    )

    self.heads.add_module(
        name = "head_5",
        module = head_upsample(
            in_channels = 1536
        )
    )

  def forward(self, x):

    output_dict = OrderedDict()

    x = self.backbone(x)

    neck_outputs = self.neck(x)

    y_pred1 = self.heads["head_1"](
        neck_outputs["ConvBNSiLU_0"],
        neck_outputs["bottleneck_0"]
    )

    output_dict["head_1"] = y_pred1

    y_pred2 = self.heads["head_2"](
        neck_outputs["ConvBNSiLU_1"],
        neck_outputs["bottleneck_1"]
    )

    output_dict["head_2"] = y_pred2

    y_pred3 = self.heads["head_3"](
        neck_outputs["bottleneck_1"]
    )

    output_dict["head_3"] = y_pred3

    y_pred4 = self.heads["head_4"](
        neck_outputs["bottleneck_0"],
        neck_outputs["bottleneck_1"]
    )

    output_dict["head_4"] = y_pred4

    y_pred5 = self.heads["head_5"](
        x,
        neck_outputs["bottleneck_0"]
    )

    output_dict["head_5"] = y_pred5

    return output_dict


def print_model_size(model):
  param_size = 0
  for param in model.parameters():
      param_size += param.nelement() * param.element_size()
  buffer_size = 0
  for buffer in model.buffers():
      buffer_size += buffer.nelement() * buffer.element_size()

  size_all_mb = (param_size + buffer_size) / 1024**2
  print('model size: {:.3f}MB'.format(size_all_mb))


In [3]:
import torch
import random
import time

a = my_yolo()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

a.to(device)

start = time.time()


x = (a(torch.rand([8,3,700,700]).to(device)))

for name, posits in x.items():
  print(f"output from {name} : {posits.shape}")

# print(f"completed yolo : result : {x.shape}")


# print(f"completed head : result : {x.shape}")

print(f"time taken to do a forward pass of the model {time.time() - start}")

print_model_size(a)

output from head_1 : torch.Size([8, 80, 4, 4])
output from head_2 : torch.Size([8, 80, 2, 2])
output from head_3 : torch.Size([8, 80, 1, 1])
output from head_4 : torch.Size([8, 80, 4, 4])
output from head_5 : torch.Size([8, 80, 8, 8])
time taken to do a forward pass of the model 34.94960045814514
model size: 220.260MB


In [1]:
import torch
from torch import nn

a = torch.rand([5])
b = torch.tenso([1,0,1,1,0])
b = nn.BCEWithLogitsLoss()
b(a)[0,:,1,1].sum()

AttributeError: module 'torch' has no attribute 'tenso'

<h1> PLAYGROUD</H1>

In [ ]:
posits

tensor([[[[-2.2934e-02, -4.0528e-02, -9.7456e-04,  ...,  1.4914e-01,
           -1.1385e-01,  7.9855e-02],
          [-2.7889e-02, -1.1845e-01, -5.5526e-02,  ..., -1.8147e-02,
            3.2890e-01,  1.7740e-03],
          [-1.9822e-02,  2.1691e-03, -5.3155e-03,  ...,  1.4366e-01,
           -4.6002e-02,  2.4464e-01],
          ...,
          [-7.2684e-02, -1.1873e-01, -4.1904e-02,  ..., -2.4458e-01,
            3.9596e-01, -1.0869e-01],
          [-2.8447e-02, -6.1854e-02, -1.1705e-01,  ...,  2.0631e-01,
            1.3241e-01,  1.7696e-01],
          [-8.4373e-03, -1.3683e-02, -6.6647e-02,  ...,  1.0852e-01,
            2.0221e-01, -1.2794e-01]],

         [[ 5.5899e-03,  4.0710e-03, -7.4751e-02,  ...,  1.8281e-02,
            9.4268e-02, -7.5644e-02],
          [ 1.4904e-02, -1.0786e-02,  3.2714e-03,  ..., -1.4564e-01,
            7.9229e-02, -1.1551e-01],
          [ 3.6557e-02,  5.6362e-02,  1.1511e-03,  ...,  7.0439e-02,
           -2.7200e-01, -2.3747e-01],
          ...,
     

In [ ]:
def dim_calc(a, n1, n2, n3, n4, n5):
  a-= (2*n1)
  print(a)
  a/=2
  print(a)
  a-= (2*n2)
  print(a)
  a/=2
  print(a)
  a-= (2*n3)
  print(a)
  a/=2
  print(a)
  a-= (2*n4)
  print(a)
  a/=2
  print(a)
  a-= (2*n5)
  print(a)

  return a


def dim_del(x1,x2,x3,x4,x5):
  dim = (2*x1) + (2**2 * x2) + (2**3 * x3) + (2**4 * x4) + (2**5 * x5)

  return dim


def find_in_dim(out_dim , n1, n2, n3, n4, n5):
  out_dim += (2*n5)
  out_dim *= 2
  out_dim += (2*n4)
  out_dim *= 2
  out_dim += (2*n3)
  out_dim *= 2
  out_dim += (2*n2)
  out_dim *= 2
  out_dim += (2*n1)
  print(f"num_layer : {n1+n2+n3+n4+n5}")
  return out_dim

In [ ]:
find_in_dim(16, 2, 2, 6, 8, 8)

num_layer : 26


700

In [ ]:
dim_calc(700, 2, 2, 6, 8, 8)

696
348.0
344.0
172.0
160.0
80.0
64.0
32.0
16.0


16.0

In [ ]:
import torch
torch.device("cuda")

device(type='cuda')

In [ ]:
import torch
import time

a = bottleneck(in_channels = 128,
               num_1x1_filters_per_rep = 64,
               num_3x3_filters_per_rep = 128,
               num_reps = 6,
               block_number = 0,
               downsample = True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

a.to(device)

start = time.time()
result = a(torch.rand([5,128,172,172]).to(device))
print(result.shape)
end = time.time()

print(f"device : {device} : time taken : {end - start}")

torch.Size([5, 128, 160, 160])
device : cuda : time taken : 0.18221807479858398


In [ ]:
# from torch import nn
import torch

b = nn.Conv2d(in_channels = 512,
              out_channels = 1024,
              kernel_size = 3,
              # padding = 1,
              # stride = 2
              )
# a(b(torch.rand([5,512,8,8]))).shape

b(torch.rand([5,512, 15, 15])).shape

torch.Size([5, 1024, 13, 13])

In [ ]:
a = seq_conv_block_1_3(
    in_channels = 100,
    num_1x1_filters_per_rep = 32,
    num_3x3_filters_per_rep = 64,
    num_reps = 800,
    block_number= 0
)

b = conv_block_1_3(
    in_channels = 100,
    num_1x1_filters_per_rep = 32,
    num_3x3_filters_per_rep = 64,
    num_reps = 800=00,
    block_number = 0
)

len(tuple(a.get_submodule("layer_block").named_children()))

In [ ]:
testmodule = conv_block_1_3(
    in_channels = 10,
    num_filters_1x1 = 32,
    num_filters_3x3 = 64,
    num_reps = 6,
    block_number = 10
)

print(tuple(testmodule.get_submodule("layer_block").named_children()))

print(testmodule(torch.ones([1,10,8,10])).shape)

(('block_10_rep_0_conv_1x1', Conv2d(10, 32, kernel_size=(1, 1), stride=(1, 1), padding=same)), ('block_10_rep_0_conv_3x3', Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)), ('block_10_rep_1_conv_1x1', Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), padding=same)), ('block_10_rep_1_conv_3x3', Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)), ('block_10_rep_2_conv_1x1', Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), padding=same)), ('block_10_rep_2_conv_3x3', Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)), ('block_10_rep_3_conv_1x1', Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), padding=same)), ('block_10_rep_3_conv_3x3', Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)), ('block_10_rep_4_conv_1x1', Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), padding=same)), ('block_10_rep_4_conv_3x3', Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)), ('block_10_rep_5_conv_1x1', Conv2d(64, 32, kernel_size=(1, 

torch.Size([1, 64, 10, 10])

In [ ]:
from torch import nn

a = nn.ModuleList()

a.append(nn.Conv2d(10,10,3))
a.append(nn.Linear(10,10))
a.append(nn.Linear(10,10))
a.append(nn.Linear(10,9))
a.register_module(name = "mr conv 1",
                  module = nn.Conv2d(12,10,3))
a.register_module(name = "mr conv 1",
                  module = nn.Conv2d(9,10,3))  #using the same name will overwrite the module
a.register_module(name = "mr conv 2",
                  module = nn.Conv2d(9,11,3))
a.register_module(name = "mr conv 2",
                  module = nn.Conv2d(9,10,3))

a.append(nn.Conv2d(9,11,3))

tuple(a.named_children())


(('0', Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))),
 ('1', Linear(in_features=10, out_features=10, bias=True)),
 ('2', Linear(in_features=10, out_features=10, bias=True)),
 ('3', Linear(in_features=10, out_features=9, bias=True)),
 ('mr conv 1', Conv2d(9, 10, kernel_size=(3, 3), stride=(1, 1))),
 ('mr conv 2', Conv2d(9, 10, kernel_size=(3, 3), stride=(1, 1))),
 ('6', Conv2d(9, 11, kernel_size=(3, 3), stride=(1, 1))))